In [1]:
import sys

# setting path
sys.path.append('..')

import pandas as pd
import copy
import uuid

from evaluation import *
from system_state import SystemState
from utils import load_problem_data

demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [2]:
def generate_unique_id():
    return str(uuid.uuid4())

In [4]:
def calculate_objective(fleet: pd.DataFrame, demand: pd.DataFrame, selling_prices: pd.DataFrame, time_step: int):
    if fleet.empty:
        return 0
    
    # fleet = fleet.merge(servers, on='server_generation')
    # fleet = fleet.merge(datacenters, on='datacenter_id')
    
    # Calculate U (utilization)
    D = get_time_step_demand(get_actual_demand(demand), time_step)
    Zf = get_capacity_by_server_generation_latency_sensitivity(fleet)
    U = get_utilization(D, Zf)

    # Calculate L (normalized lifespan)
    L = get_normalized_lifespan(fleet)

    # Calculate P (profit)
    selling_prices = change_selling_prices_format(selling_prices)
    P = get_profit(D, 
                   Zf, 
                   selling_prices,
                   fleet)

    return U * L * P

In [ ]:
# def generate_possible_actions(state, servers_info):
#     buy_actions = []
#     other_actions = []
    
#     # Consider buying new servers
#     for dc in state.datacenter_capacity.itertuples():
#         for server in servers_info.itertuples():
#             if state.time_step in eval(server.release_time):
#                 if dc.slots_capacity - dc.used_slots >= server.slots_size:
#                     buy_actions.append({
#                         'action': 'buy',
#                         'datacenter_id': dc.datacenter_id,
#                         'server_generation': server.server_generation,
#                         'server_id': None  # We'll generate IDs when actually buying
#                     })
    
#     # Consider dismissing servers
#     for server in state.fleet.itertuples():
#         other_actions.append({
#             'action': 'dismiss',
#             'datacenter_id': server.datacenter_id,
#             'server_generation': server.server_generation,
#             'server_id': server.server_id
#         })
    
#     # Consider moving servers
#     for server in state.fleet.itertuples():
#         for dc in state.datacenter_capacity.itertuples():
#             if dc.datacenter_id != server.datacenter_id:
#                 if dc.slots_capacity - dc.used_slots >= servers_info.loc[servers_info['server_generation'] == server.server_generation, 'slots_size'].values[0]:
#                     other_actions.append({
#                         'action': 'move',
#                         'datacenter_id': dc.datacenter_id,
#                         'server_generation': server.server_generation,
#                         'server_id': server.server_id
#                     })
    
#     return buy_actions, other_actions

In [5]:
# def greedy_decisions(state, demand, servers_info, selling_prices, datacenters):
#     decisions = []
#     buy_actions, other_actions = generate_possible_actions(state, servers_info)
    
#     # First, handle non-buy actions
#     while other_actions:
#         best_action = None
#         best_objective = calculate_objective(state.fleet, demand, selling_prices, state.time_step)
        
#         for action in other_actions:
#             temp_state = copy.deepcopy(state)
#             temp_state.update_state([action])
#             new_objective = calculate_objective(temp_state.fleet, demand, selling_prices, state.time_step)
            
#             if new_objective > best_objective:
#                 best_objective = new_objective
#                 best_action = action
        
#         if best_action:
#             decisions.append(best_action)
#             state.update_state([best_action])
#             other_actions = [a for a in other_actions if a['server_id'] != best_action['server_id']]
#         else:
#             break
    
#     # Now, handle buy actions
#     while buy_actions:
#         best_action = None
#         best_objective = calculate_objective(state.fleet, demand, selling_prices, state.time_step)
#         best_count = 0
        
#         for action in buy_actions:
#             dc_id = action['datacenter_id']
#             dc_capacity = datacenters.loc[datacenters['datacenter_id'] == dc_id, 'slots_capacity'].values[0]
#             dc_used_slots = state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'].values[0]
#             server_size = servers_info.loc[servers_info['server_generation'] == action['server_generation'], 'slots_size'].values[0]
            
#             max_possible_servers = (dc_capacity - dc_used_slots) // server_size
            
#             for count in range(1, max_possible_servers + 1):
#                 temp_actions = [{**action, 'server_id': generate_unique_id()} for _ in range(count)]
                
#                 temp_state = copy.deepcopy(state)
#                 temp_state.update_state(temp_actions)
#                 new_objective = calculate_objective(temp_state.fleet, demand, selling_prices, state.time_step)
                
#                 if new_objective > best_objective:
#                     best_objective = new_objective
#                     best_action = action
#                     best_count = count
        
#         if best_action:
#             new_purchases = [{**best_action, 'server_id': generate_unique_id()} for _ in range(best_count)]
#             decisions.extend(new_purchases)
#             state.update_state(new_purchases)
            
#             # Update datacenter capacity
#             dc_id = best_action['datacenter_id']
#             server_size = servers_info.loc[servers_info['server_generation'] == best_action['server_generation'], 'slots_size'].values[0]
#             state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'] += best_count * server_size
            
#             # Remove the action if datacenter is full
#             if state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'].values[0] == datacenters.loc[datacenters['datacenter_id'] == dc_id, 'slots_capacity'].values[0]:
#                 buy_actions = [a for a in buy_actions if a['datacenter_id'] != dc_id]
#         else:
#             break
    
#     return decisions

In [6]:
state = SystemState(datacenters, servers)